**3. Сверточная нейронная сеть**

In [52]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist   
from keras.models import Sequential
from keras.utils import np_utils
from keras.layers.core import Activation
from keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D
 
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
 
# Стандартизируем входные данные
x_train = x_train / 255
x_test = x_test / 255
 
# Преобразовываем векторы классов в бинарные матрицы классов 
y_train = np_utils.to_categorical(y_train, 10)
y_test = np_utils.to_categorical(y_test, 10)

# Добавляем еще одно измерение (одну ось) для цветовой компоненты
x_train = np.expand_dims(x_train, axis=3)
x_test = np.expand_dims(x_test, axis=3)

# Строим модель
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(10,  activation='softmax'))
model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_62 (Conv2D)          (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_58 (MaxPoolin  (None, 14, 14, 32)       0         
 g2D)                                                            
                                                                 
 flatten_18 (Flatten)        (None, 6272)              0         
                                                                 
 dense_36 (Dense)            (None, 128)               802944    
                                                                 
 dense_37 (Dense)            (None, 10)                1290      
                                                                 
Total params: 804,554
Trainable params: 804,554
Non-trainable params: 0
_______________________________________________

In [53]:
# Компилируем модель
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# Подключаем модуль для построения графиков качества модели на валидационной и тренировочной выборке от количества эпох
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0,
                          write_graph=True, write_images=False)

# Обучаем модель 
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split=0.2, callbacks=[tensorboard])

Epoch 1/5
1500/1500 [==============================] - 38s 25ms/step - loss: 0.3999 - accuracy: 0.8573 - val_loss: 0.2952 - val_accuracy: 0.8928
Epoch 2/5
1500/1500 [==============================] - 39s 26ms/step - loss: 0.2655 - accuracy: 0.9044 - val_loss: 0.2662 - val_accuracy: 0.9023
Epoch 3/5
1500/1500 [==============================] - 37s 25ms/step - loss: 0.2208 - accuracy: 0.9179 - val_loss: 0.2632 - val_accuracy: 0.9055
Epoch 4/5
1500/1500 [==============================] - 35s 23ms/step - loss: 0.1849 - accuracy: 0.9313 - val_loss: 0.2560 - val_accuracy: 0.9075
Epoch 5/5
1500/1500 [==============================] - 35s 23ms/step - loss: 0.1587 - accuracy: 0.9402 - val_loss: 0.2522 - val_accuracy: 0.9119


In [54]:
# Проверяем качество на тестовых данных
model.evaluate(x_test, y_test)

313/313 [==============================] - 2s 7ms/step - loss: 0.2713 - accuracy: 0.9101


[0.2712896764278412, 0.910099983215332]

In [ ]:
# Строим графики качества модели на валидационной и тренировочной выборке от количества эпох
%load_ext tensorboard
%tensorboard --logdir logs

**Вывод:**

Качество классификации по accuracy составило 94.19% на тренировочной выборке и 90.96% на тестовой. Данный результат выше почти на 4% по сравнению с полносвязанной нейронной сетью. К тому же, количество итераций уменьшилось в 10 раз. С 50 до 5.

**Добавляем +2 сверточных слоя**

In [ ]:
from tensorflow.python.keras.layers.core import Activation
# Строим модель
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=(28, 28, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3,3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))
model.summary()


In [40]:
# Компилируем модель
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# Обучаем модель 
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split=0.2,)

Epoch 1/5
1500/1500 [==============================] - 100s 66ms/step - loss: 0.4506 - accuracy: 0.8336 - val_loss: 0.3349 - val_accuracy: 0.8794
Epoch 2/5
1500/1500 [==============================] - 97s 65ms/step - loss: 0.2807 - accuracy: 0.8965 - val_loss: 0.3042 - val_accuracy: 0.8868
Epoch 3/5
1500/1500 [==============================] - 97s 65ms/step - loss: 0.2346 - accuracy: 0.9129 - val_loss: 0.2487 - val_accuracy: 0.9072
Epoch 4/5
1500/1500 [==============================] - 98s 65ms/step - loss: 0.2046 - accuracy: 0.9247 - val_loss: 0.2347 - val_accuracy: 0.9121
Epoch 5/5
1500/1500 [==============================] - 97s 65ms/step - loss: 0.1750 - accuracy: 0.9342 - val_loss: 0.2257 - val_accuracy: 0.9159


In [41]:
# Проверяем качество на тестовых данных
model.evaluate(x_test, y_test)

313/313 [==============================] - 6s 18ms/step - loss: 0.2467 - accuracy: 0.9110


[0.24665500223636627, 0.9110000133514404]

**Вывод**

Как видим - добавление 2 дополнительных сверточных слоев повысило качество классификкации модели, но не сильно: не более чем на 1%

**Добавляем Batch Normalization**

In [ ]:
from tensorflow.python.keras.layers import BatchNormalization
# Строим модель
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', input_shape=(28, 28, 1)))
BatchNormalization()
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3,3), padding='same'))
BatchNormalization()
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3,3), padding='same'))
BatchNormalization()
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(512))
BatchNormalization()
model.add(Activation('relu'))
model.add(Dense(10))
BatchNormalization()
model.add(Activation('softmax'))
model.summary()

In [47]:
# Компилируем модель
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

# Обучаем модель 
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_split=0.2)

Epoch 1/5
1500/1500 [==============================] - 98s 65ms/step - loss: 0.4356 - accuracy: 0.8397 - val_loss: 0.3251 - val_accuracy: 0.8759
Epoch 2/5
1500/1500 [==============================] - 96s 64ms/step - loss: 0.2779 - accuracy: 0.8986 - val_loss: 0.2781 - val_accuracy: 0.8953
Epoch 3/5
1500/1500 [==============================] - 96s 64ms/step - loss: 0.2317 - accuracy: 0.9130 - val_loss: 0.2404 - val_accuracy: 0.9135
Epoch 4/5
1500/1500 [==============================] - 96s 64ms/step - loss: 0.1955 - accuracy: 0.9280 - val_loss: 0.2277 - val_accuracy: 0.9149
Epoch 5/5
1500/1500 [==============================] - 96s 64ms/step - loss: 0.1688 - accuracy: 0.9363 - val_loss: 0.2344 - val_accuracy: 0.9191


In [48]:
# Проверяем качество на тестовых данных
model.evaluate(x_test, y_test)

313/313 [==============================] - 6s 18ms/step - loss: 0.2566 - accuracy: 0.9124


[0.2565687894821167, 0.9124000072479248]

**Вывод**

Использование BatchNormalization повысило качество работы сверточной нейросети, но не очень сильно. Это объясняется тем, что с увеличением количества сверточных слоев - модель переусложняется и качество работы растет незначительно.
*******************************************************************************

Если сравнивать сверточную сеть с полносвязанной, то работа сверточной сети на настоящих данных дает заметно лучший результат. Это объясняется тем, что сверточные сети гораздо лучше работают в области распознавания изображений